In [ ]:
!python -m pip install --user --upgrade pip

     |████████████████████████████████| 1.5MB 7.7MB/s 
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [ ]:
!pip3 install kfp --upgrade --user

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
     |████████████████████████████████| 159 kB 8.4 MB/s 
     |████████████████████████████████| 636 kB 11.0 MB/s 
     |████████████████████████████████| 1.5 MB 16.2 MB/s 
     |████████████████████████████████| 54 kB 2.3 MB/s 
     |████████████████████████████████| 50 kB 5.0 MB/s 
     |████████████████████████████████| 56 kB 3.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 87 kB 5.6 MB/s 
     |████████████████████████████████| 1.0 MB 36.8 MB/s 
     |████████████████████████████████| 200 kB 49.8 MB/s 
  Created wheel for kfp: filename=kfp-1.4.0-py3-none-any.whl size=221064 sha256=9d68fe4be50ff07f628a20aec916ea35cd1d83f5f3cf5bfe391d6dcd25d7bf58
  Stored in directory: /root/.c

In [ ]:
import kfp
from kfp import dsl
import kfp.components as comp

In [ ]:
def load_data_op():
    return dsl.ContainerOp(
        name = 'Load Data',
        image = 'mavencodevv/load_cm:v.0.2',
        arguments = [],
        file_outputs={
            'data': '/load_data/data'
        }      
    )

In [ ]:
def preprocess_op(data):
    return dsl.ContainerOp(
        name = 'Preprocess Data',
        image = 'mavencodevv/preprocess_cm:v.0.1',
        arguments = ['--data', data],
        file_outputs={
            'clean_data': '/preprocess/clean_data'   
        }
    )

In [ ]:
def autoencoder_op(clean_data):
    return dsl.ContainerOp(
        name = 'Autoencoder model',
        image = 'mavencodevv/train_cm:v.0.3',
        arguments = ['--clean_data', clean_data
            ],
        file_outputs={
            'fit_model': '/train_keras/fit_model' 
        }
    )

In [ ]:
def pca_op(clean_data):
  return dsl.ContainerOp(
      name = "PCA model",
      image = "mavencodevv/pca_cm:v.0.1",
      arguments = ['--clean_data', clean_data],
      file_outputs = {'pca_metrics': '/pca/pca_metrics'}
  )

In [ ]:
def test_op(clean_data, fit_model):
    return dsl.ContainerOp(
        name = 'Test autoencoder',
        image = 'mavencodevv/test_cm:v.0.4',
        arguments = ['--clean_data', clean_data,
                    '--fit_model', fit_model
            ],
        file_outputs={
            'metrics': '/test/metrics' 
        }
    )

In [ ]:
def results(metrics, pca_metrics):
    return dsl.ContainerOp(
        name = 'Results',
        image = 'mavencodevv/results_cm:v.0.1',
        arguments = ['--metrics', metrics,
                    '--pca_metrics', pca_metrics
            ] 
        
    )

In [ ]:
@dsl.pipeline(
    name='Condition monitoring pipeline',
   description='An ML reusable pipeline that monitors the wear and tear of a Factory machine'
)

# Define parameters to be fed into pipeline
def cm_pipeline(): 
    
    _load_data_op = load_data_op()
    
    _preprocess_op = preprocess_op(
        dsl.InputArgumentPath(_load_data_op.outputs['data'])).after(_load_data_op)
    
    _autoencoder_op = autoencoder_op(
        dsl.InputArgumentPath(_preprocess_op.outputs['clean_data'])).after(_preprocess_op)
    _pca_op = pca_op(
        dsl.InputArgumentPath(_preprocess_op.outputs['clean_data'])).after(_preprocess_op)
    _test_op = test_op(
        dsl.InputArgumentPath(_preprocess_op.outputs['clean_data']),
        dsl.InputArgumentPath(_autoencoder_op.outputs['fit_model'])).after(_autoencoder_op)
    _results_op = results(
        dsl.InputArgumentPath(_test_op.outputs['metrics']),
        dsl.InputArgumentPath(_pca_op.outputs['pca_metrics'])).after(_test_op,_pca_op)
    

In [ ]:
# Compile pipeline to generate compressed YAML definition of the pipeline.
experiment_name = 'cm_pipeline'

kfp.compiler.Compiler().compile(cm_pipeline, '{}.yaml'.format(experiment_name))

/root/.local/lib/python3.7/site-packages/kfp/dsl/_container_op.py:1039: FutureWarning: Please create reusable components instead of constructing ContainerOp instances directly. Reusable components are shareable, portable and have compatibility and support guarantees. Please see the documentation: https://www.kubeflow.org/docs/pipelines/sdk/component-development/#writing-your-component-definition-file The components can be created manually (or, in case of python, using kfp.components.create_component_from_func or func_to_container_op) and then loaded using kfp.components.load_component_from_file, load_component_from_uri or load_component_from_text: https://kubeflow-pipelines.readthedocs.io/en/stable/source/kfp.components.html#kfp.components.load_component_from_file
  category=FutureWarning,


In [ ]:
client = kfp.Client()
client.create_run_from_pipeline_func(cm_pipeline, arguments={})